# Imports

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
from numpy.linalg import inv

In [ ]:
def obj (x) :    
    # return np.power(x, 4) + np.power(x, 3) - 10*np.square(x) + x + 5
    return 0.025*np.square(x) + np.sin(x)    

objkey = lambda x : obj(x.reshape(-1, 1))[0]

# Initialisation

In [ ]:
llim = -20
rlim = 20
lin = np.linspace(llim, rlim, 100)
noParticles = 25
x_min = -1.49593
w = 0.75
c1 = 2
c2 = 2
vmax = 0.4

eps = 1e-2
xs = eps*(2*np.random.rand(noParticles, 1) - 1) + x_min
vs = eps*np.random.rand(noParticles, 1)
# vs *= np.where(xs - x_min <= 0, 1, -1)

pbest = np.copy(xs)
gbest = min(pbest, key = lambda x : obj(x.reshape(1,-1))[0])

pbest_past = np.array([
    4.5 if x > x_min else -7.5 for x in xs
]).reshape(-1,1)

xs_init = np.copy(xs)

## Reverse-PSO Loop

In [ ]:
noIter = 100
less_once = False
foreigns = None
step = False
for i in range(0, noIter) :
    
    if step :
        plt.plot(lin, obj(lin.reshape(-1, 1)))
        plt.plot (xs, obj(xs), 'ro', markersize=3)
        plt.pause(0.0001)
        input()
    
    r1s = np.random.rand (noParticles, 1)
    r2s = np.random.rand (noParticles, 1)
    
    
    if not less_once :
        which_pbest = xs
        gb = x_min
    else :
        if foreigns is None :
            foreigns = np.array([
                4.5 if x > x_min else -7.5 for x in xs
            ]).reshape(-1,1)
        which_pbest = foreigns
        gb = gbest[0]
        
    mats = [np.array([
            [1 - c1*r1 - c2*r2, w, c1*r1*pb + c2*r2*gb], 
            [-c1*r1 - c2*r2, w, c1*r1*pb + c2*r2*gb], 
            [0, 0, 1]
        ]) for r1, r2, pb in zip(r1s[:,0], r2s[:,0], which_pbest[:,0])]

    vecs = np.array ([np.dot (inv(m), np.array([x, v, 1])) for m, x, v in zip (mats, xs[:,0], vs[:,0])])    
    vs = vecs[:,1].reshape(-1, 1)
    sgn = vs < 0
    vs = np.minimum(vmax, np.abs(vs))
    vs[sgn] *= -1    
    xs -= vs
    
    less = obj(xs) <= obj(pbest)
    xs[less] = np.copy(pbest[less])
    more = np.invert(less)
    pbest[more] = np.copy(xs[more])
    gbest = min(pbest, key=lambda x : obj(x.reshape(1,-1))[0])
    
    if less.all() :
        if not less_once :
            less_once = True
        else :
            break
    
if step :
    plt.plot(lin, obj(lin.reshape(-1, 1)))
    plt.plot (xs, obj(xs), 'ro', markersize=3)
    plt.pause(0.0001)
    input()

print(i)
pass

## Plot Final Distribution

In [ ]:
plt.plot(lin, obj(lin.reshape(-1, 1)))
plt.plot (xs, obj(xs), 'ro', markersize=3)